In [3]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False
import gc


# 인코더 추가
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
#상수항추가
from statsmodels.tools.tools import add_constant
# 카이제곱, ANOVA
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
#Turkeyhsd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

### 데이터 불러오기

In [12]:
# parquet 파일 데이터를 읽어온다.
# x 값으로 쓸 데이터 
df1_train = pd.read_parquet('open/train/3.승인매출정보/201807_train_승인매출정보.parquet')
df2_train = pd.read_parquet('open/train/3.승인매출정보/201808_train_승인매출정보.parquet')
df3_train = pd.read_parquet('open/train/3.승인매출정보/201809_train_승인매출정보.parquet')
df4_train = pd.read_parquet('open/train/3.승인매출정보/201810_train_승인매출정보.parquet')
df5_train = pd.read_parquet('open/train/3.승인매출정보/201811_train_승인매출정보.parquet')
df6_train = pd.read_parquet('open/train/3.승인매출정보/201812_train_승인매출정보.parquet')

In [14]:
# 결과데이터를 불러온다. 
segment_df = pd.read_csv('Segment.csv')
segment_df

,기준년월,ID,Segment
0,201807,TRAIN_000000,D
1,201807,TRAIN_000001,E
2,201807,TRAIN_000002,C
3,201807,TRAIN_000003,D
4,201807,TRAIN_000004,E
...,...,...,...
2399995,201812,TRAIN_399995,E
2399996,201812,TRAIN_399996,D
2399997,201812,TRAIN_399997,C
2399998,201812,TRAIN_399998,E


In [18]:
# 데이터프레임의 컬럼 이름을 리스트에 담는다. 
column_list = df1_train.columns.tolist()

In [20]:
print(column_list)

['기준년월', 'ID', '최종이용일자_기본', '최종이용일자_신판', '최종이용일자_CA', '최종이용일자_카드론', '최종이용일자_체크', '최종이용일자_일시불', '최종이용일자_할부', '이용건수_신용_B0M', '이용건수_신판_B0M', '이용건수_일시불_B0M', '이용건수_할부_B0M', '이용건수_할부_유이자_B0M', '이용건수_할부_무이자_B0M', '이용건수_부분무이자_B0M', '이용건수_CA_B0M', '이용건수_체크_B0M', '이용건수_카드론_B0M', '이용금액_일시불_B0M', '이용금액_할부_B0M', '이용금액_할부_유이자_B0M', '이용금액_할부_무이자_B0M', '이용금액_부분무이자_B0M', '이용금액_CA_B0M', '이용금액_체크_B0M', '이용금액_카드론_B0M', '이용후경과월_신용', '이용후경과월_신판', '이용후경과월_일시불', '이용후경과월_할부', '이용후경과월_할부_유이자', '이용후경과월_할부_무이자', '이용후경과월_부분무이자', '이용후경과월_CA', '이용후경과월_체크', '이용후경과월_카드론', '이용건수_신용_R12M', '이용건수_신판_R12M', '이용건수_일시불_R12M', '이용건수_할부_R12M', '이용건수_할부_유이자_R12M', '이용건수_할부_무이자_R12M', '이용건수_부분무이자_R12M', '이용건수_CA_R12M', '이용건수_체크_R12M', '이용건수_카드론_R12M', '이용금액_일시불_R12M', '이용금액_할부_R12M', '이용금액_할부_유이자_R12M', '이용금액_할부_무이자_R12M', '이용금액_부분무이자_R12M', '이용금액_CA_R12M', '이용금액_체크_R12M', '이용금액_카드론_R12M', '최대이용금액_일시불_R12M', '최대이용금액_할부_R12M', '최대이용금액_할부_유이자_R12M', '최대이용금액_할부_무이자_R12M', '최대이용금액_부분무이자_R12M', '최대이용금액_CA_R12M', '최대이용금액_체크_R12M', '

In [22]:
# 슬라이싱
column_list = column_list[129:223]

In [24]:
print(column_list)

['이용가맹점수', '이용금액_해외', '쇼핑_도소매_이용금액', '쇼핑_백화점_이용금액', '쇼핑_마트_이용금액', '쇼핑_슈퍼마켓_이용금액', '쇼핑_편의점_이용금액', '쇼핑_아울렛_이용금액', '쇼핑_온라인_이용금액', '쇼핑_기타_이용금액', '교통_주유이용금액', '교통_정비이용금액', '교통_통행료이용금액', '교통_버스지하철이용금액', '교통_택시이용금액', '교통_철도버스이용금액', '여유_운동이용금액', '여유_Pet이용금액', '여유_공연이용금액', '여유_공원이용금액', '여유_숙박이용금액', '여유_여행이용금액', '여유_항공이용금액', '여유_기타이용금액', '납부_통신비이용금액', '납부_관리비이용금액', '납부_렌탈료이용금액', '납부_가스전기료이용금액', '납부_보험료이용금액', '납부_유선방송이용금액', '납부_건강연금이용금액', '납부_기타이용금액', '_1순위업종', '_1순위업종_이용금액', '_2순위업종', '_2순위업종_이용금액', '_3순위업종', '_3순위업종_이용금액', '_1순위쇼핑업종', '_1순위쇼핑업종_이용금액', '_2순위쇼핑업종', '_2순위쇼핑업종_이용금액', '_3순위쇼핑업종', '_3순위쇼핑업종_이용금액', '_1순위교통업종', '_1순위교통업종_이용금액', '_2순위교통업종', '_2순위교통업종_이용금액', '_3순위교통업종', '_3순위교통업종_이용금액', '_1순위여유업종', '_1순위여유업종_이용금액', '_2순위여유업종', '_2순위여유업종_이용금액', '_3순위여유업종', '_3순위여유업종_이용금액', '_1순위납부업종', '_1순위납부업종_이용금액', '_2순위납부업종', '_2순위납부업종_이용금액', '_3순위납부업종', '_3순위납부업종_이용금액', '할부건수_3M_R12M', '할부건수_6M_R12M', '할부건수_12M_R12M', '할부건수_14M_R12M', '할부금액_3M_R12M', '할부금액_6M_R12M', '할부금액_12M_R12M', '할부금액_14M_R12M', 

In [26]:
df1_train = df1_train[column_list]

In [28]:
df2_train = df2_train[column_list]
df3_train = df3_train[column_list]
df4_train = df4_train[column_list]
df5_train = df5_train[column_list]
df6_train = df6_train[column_list]

In [30]:
combined_df = pd.concat([df1_train, df2_train, df3_train, df4_train, df5_train, df6_train], 
                         axis=0,        # 행 방향
                         ignore_index=True)  # 인덱스 초기화

In [34]:
# 세그먼트와 데이터를 합쳐준다. 

all_df = pd.concat([segment_df,combined_df], axis = 1)
all_df

,기준년월,ID,Segment,이용가맹점수,이용금액_해외,쇼핑_도소매_이용금액,쇼핑_백화점_이용금액,쇼핑_마트_이용금액,쇼핑_슈퍼마켓_이용금액,쇼핑_편의점_이용금액,...,할부금액_무이자_12M_R12M,할부금액_무이자_14M_R12M,할부건수_부분_3M_R12M,할부건수_부분_6M_R12M,할부건수_부분_12M_R12M,할부건수_부분_14M_R12M,할부금액_부분_3M_R12M,할부금액_부분_6M_R12M,할부금액_부분_12M_R12M,할부금액_부분_14M_R12M
0,201807,TRAIN_000000,D,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,201807,TRAIN_000001,E,32,0,645,0,0,435,315,...,0,0,0,0,0,0,0,0,0,0
2,201807,TRAIN_000002,C,27,0,1038,751,924,504,239,...,0,0,0,0,0,0,0,0,0,0
3,201807,TRAIN_000003,D,10,0,0,0,801,487,0,...,0,0,0,0,0,0,0,0,0,0
4,201807,TRAIN_000004,E,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,E,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2399996,201812,TRAIN_399996,D,15,0,1496,0,0,714,0,...,0,0,0,0,0,0,0,0,0,0
2399997,201812,TRAIN_399997,C,27,0,0,0,0,400,300,...,0,0,0,0,0,0,0,0,0,0
2399998,201812,TRAIN_399998,E,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
print(all_df.isna().sum().to_string())

기준년월                       0
ID                         0
Segment                    0
이용가맹점수                     0
이용금액_해외                    0
쇼핑_도소매_이용금액                0
쇼핑_백화점_이용금액                0
쇼핑_마트_이용금액                 0
쇼핑_슈퍼마켓_이용금액               0
쇼핑_편의점_이용금액                0
쇼핑_아울렛_이용금액                0
쇼핑_온라인_이용금액                0
쇼핑_기타_이용금액                 0
교통_주유이용금액                  0
교통_정비이용금액                  0
교통_통행료이용금액                 0
교통_버스지하철이용금액               0
교통_택시이용금액                  0
교통_철도버스이용금액                0
여유_운동이용금액                  0
여유_Pet이용금액                 0
여유_공연이용금액                  0
여유_공원이용금액                  0
여유_숙박이용금액                  0
여유_여행이용금액                  0
여유_항공이용금액                  0
여유_기타이용금액                  0
납부_통신비이용금액                 0
납부_관리비이용금액                 0
납부_렌탈료이용금액                 0
납부_가스전기료이용금액               0
납부_보험료이용금액                 0
납부_유선방송이용금액                0
납부_건강연금이용금액                0
납부_기타이용금액     

_1순위업종                539992
_2순위업종                912725
_3순위업종               1107898
_1순위쇼핑업종              922663
_2순위쇼핑업종             1135042
_3순위쇼핑업종             1312267
_1순위교통업종             1164494
_2순위교통업종             1656423
_3순위교통업종             2045455
_1순위여유업종             1987260
_2순위여유업종             2302286
_3순위여유업종             2377725
_1순위납부업종             1216263
_2순위납부업종             2033640
_3순위납부업종             2310187

In [50]:
for col in column_list:
    try:
        var_value = all_df[col].var()
        print(f"{col}: {var_value:.2f}")
    except TypeError:
        print(f"❌ 분산 계산 불가 (문자열 포함): {col}")


이용가맹점수: 620.33
이용금액_해외: 244947.96
쇼핑_도소매_이용금액: 544484.29
쇼핑_백화점_이용금액: 77740.47
쇼핑_마트_이용금액: 338766.64
쇼핑_슈퍼마켓_이용금액: 228902.32
쇼핑_편의점_이용금액: 114447.16
쇼핑_아울렛_이용금액: 21936.57
쇼핑_온라인_이용금액: 18827604.18
쇼핑_기타_이용금액: 21235.64
교통_주유이용금액: 1163692.50
교통_정비이용금액: 38359.46
교통_통행료이용금액: 2.75
교통_버스지하철이용금액: 128310.55
교통_택시이용금액: 13153.34
교통_철도버스이용금액: 18573.09
여유_운동이용금액: 99932.42
여유_Pet이용금액: 3211.95
여유_공연이용금액: 2226.47
여유_공원이용금액: 103.48
여유_숙박이용금액: 3802.57
여유_여행이용금액: 0.00
여유_항공이용금액: 18086.05
여유_기타이용금액: 1654.66
납부_통신비이용금액: 1110276.92
납부_관리비이용금액: 656954.84
납부_렌탈료이용금액: 0.00
납부_가스전기료이용금액: 7593.75
납부_보험료이용금액: 423363.85
납부_유선방송이용금액: 0.00
납부_건강연금이용금액: 0.00
납부_기타이용금액: 8414.30
❌ 분산 계산 불가 (문자열 포함): _1순위업종
_1순위업종_이용금액: 78376831.02
❌ 분산 계산 불가 (문자열 포함): _2순위업종
_2순위업종_이용금액: 8171160.65
❌ 분산 계산 불가 (문자열 포함): _3순위업종
_3순위업종_이용금액: 2717267.15
❌ 분산 계산 불가 (문자열 포함): _1순위쇼핑업종
_1순위쇼핑업종_이용금액: 18248816.94
❌ 분산 계산 불가 (문자열 포함): _2순위쇼핑업종
_2순위쇼핑업종_이용금액: 441931.88
❌ 분산 계산 불가 (문자열 포함): _3순위쇼핑업종
_3순위쇼핑업종_이용금액: 256998.47
❌ 분산 계산 불가 (문자열 포함): _1

In [125]:
all_df['할부금액_부분_3M_R12M'].value_counts()

할부금액_부분_3M_R12M
0    2400000
Name: count, dtype: int64

In [52]:
all_df['여유_여행이용금액'].value_counts()

여유_여행이용금액
0    2400000
Name: count, dtype: int64

- 분산이 0인 것들은 [할부건수_14M_R12M, 할부건수_유이자_14M_R12M, 할부건수_부분_12M_R12M, 할부건수_무이자_14M_R12M] 컬럼을 제외하면
- 모두 0 하나만을 데이터로 가지고 있다.
- [여유_여행이용금액, 납부_렌탈료이용금액, 납부_유선방송이용금액, 납부_건강연금이용금액, 할부건수_부분_3M_R12M, 할부건수_부분_6M_R12M, 할부건수_부분_14M_R12M, 할부금액_부분_3M_R12M]의 경우 0 하나만을 가진 데이터값이다. 

- 0 하나만 가진 데이터값은 상관관계 분석이나 추후 학습에 방해되므로 drop

In [88]:
all_df.drop(['여유_여행이용금액', '납부_렌탈료이용금액', '납부_유선방송이용금액', 
             '납부_건강연금이용금액', '할부건수_부분_3M_R12M', '할부건수_부분_6M_R12M', '할부건수_부분_14M_R12M'],
            axis=1, inplace=True)


In [128]:

all_df.drop(['할부금액_부분_3M_R12M'],
            axis=1, inplace=True)


### 변수간 관계파악
Anova -> ETA제곱
카이제곱 -> 크레이머스V

In [83]:
print(column_list)

['이용가맹점수', '이용금액_해외', '쇼핑_도소매_이용금액', '쇼핑_백화점_이용금액', '쇼핑_마트_이용금액', '쇼핑_슈퍼마켓_이용금액', '쇼핑_편의점_이용금액', '쇼핑_아울렛_이용금액', '쇼핑_온라인_이용금액', '쇼핑_기타_이용금액', '교통_주유이용금액', '교통_정비이용금액', '교통_통행료이용금액', '교통_버스지하철이용금액', '교통_택시이용금액', '교통_철도버스이용금액', '여유_운동이용금액', '여유_Pet이용금액', '여유_공연이용금액', '여유_공원이용금액', '여유_숙박이용금액', '여유_여행이용금액', '여유_항공이용금액', '여유_기타이용금액', '납부_통신비이용금액', '납부_관리비이용금액', '납부_렌탈료이용금액', '납부_가스전기료이용금액', '납부_보험료이용금액', '납부_유선방송이용금액', '납부_건강연금이용금액', '납부_기타이용금액', '_1순위업종', '_1순위업종_이용금액', '_2순위업종', '_2순위업종_이용금액', '_3순위업종', '_3순위업종_이용금액', '_1순위쇼핑업종', '_1순위쇼핑업종_이용금액', '_2순위쇼핑업종', '_2순위쇼핑업종_이용금액', '_3순위쇼핑업종', '_3순위쇼핑업종_이용금액', '_1순위교통업종', '_1순위교통업종_이용금액', '_2순위교통업종', '_2순위교통업종_이용금액', '_3순위교통업종', '_3순위교통업종_이용금액', '_1순위여유업종', '_1순위여유업종_이용금액', '_2순위여유업종', '_2순위여유업종_이용금액', '_3순위여유업종', '_3순위여유업종_이용금액', '_1순위납부업종', '_1순위납부업종_이용금액', '_2순위납부업종', '_2순위납부업종_이용금액', '_3순위납부업종', '_3순위납부업종_이용금액', '할부건수_3M_R12M', '할부건수_6M_R12M', '할부건수_12M_R12M', '할부건수_14M_R12M', '할부금액_3M_R12M', '할부금액_6M_R12M', '할부금액_12M_R12M', '할부금액_14M_R12M', 

In [90]:
# 크레이머스V와 eta를 구하기 위한 함수
def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(k, r) - 1)))

def eta_squared(anova_ss_between, total_ss):
    return anova_ss_between / total_ss if total_ss != 0 else np.nan

In [92]:
# 수치형/범주형 자동 구분
num_cols = all_df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = all_df.select_dtypes(exclude=[np.number]).columns.tolist()

In [94]:
print(cat_cols)

['ID', 'Segment', '_1순위업종', '_2순위업종', '_3순위업종', '_1순위쇼핑업종', '_2순위쇼핑업종', '_3순위쇼핑업종', '_1순위교통업종', '_2순위교통업종', '_3순위교통업종', '_1순위여유업종', '_2순위여유업종', '_3순위여유업종', '_1순위납부업종', '_2순위납부업종', '_3순위납부업종']


In [98]:
del cat_cols[0]

In [102]:
print(cat_cols)

['Segment', '_1순위업종', '_2순위업종', '_3순위업종', '_1순위쇼핑업종', '_2순위쇼핑업종', '_3순위쇼핑업종', '_1순위교통업종', '_2순위교통업종', '_3순위교통업종', '_1순위여유업종', '_2순위여유업종', '_3순위여유업종', '_1순위납부업종', '_2순위납부업종', '_3순위납부업종']


In [104]:
print(num_cols)

['기준년월', '이용가맹점수', '이용금액_해외', '쇼핑_도소매_이용금액', '쇼핑_백화점_이용금액', '쇼핑_마트_이용금액', '쇼핑_슈퍼마켓_이용금액', '쇼핑_편의점_이용금액', '쇼핑_아울렛_이용금액', '쇼핑_온라인_이용금액', '쇼핑_기타_이용금액', '교통_주유이용금액', '교통_정비이용금액', '교통_통행료이용금액', '교통_버스지하철이용금액', '교통_택시이용금액', '교통_철도버스이용금액', '여유_운동이용금액', '여유_Pet이용금액', '여유_공연이용금액', '여유_공원이용금액', '여유_숙박이용금액', '여유_항공이용금액', '여유_기타이용금액', '납부_통신비이용금액', '납부_관리비이용금액', '납부_가스전기료이용금액', '납부_보험료이용금액', '납부_기타이용금액', '_1순위업종_이용금액', '_2순위업종_이용금액', '_3순위업종_이용금액', '_1순위쇼핑업종_이용금액', '_2순위쇼핑업종_이용금액', '_3순위쇼핑업종_이용금액', '_1순위교통업종_이용금액', '_2순위교통업종_이용금액', '_3순위교통업종_이용금액', '_1순위여유업종_이용금액', '_2순위여유업종_이용금액', '_3순위여유업종_이용금액', '_1순위납부업종_이용금액', '_2순위납부업종_이용금액', '_3순위납부업종_이용금액', '할부건수_3M_R12M', '할부건수_6M_R12M', '할부건수_12M_R12M', '할부건수_14M_R12M', '할부금액_3M_R12M', '할부금액_6M_R12M', '할부금액_12M_R12M', '할부금액_14M_R12M', '할부건수_유이자_3M_R12M', '할부건수_유이자_6M_R12M', '할부건수_유이자_12M_R12M', '할부건수_유이자_14M_R12M', '할부금액_유이자_3M_R12M', '할부금액_유이자_6M_R12M', '할부금액_유이자_12M_R12M', '할부금액_유이자_14M_R12M', '할부건수_무이자_3M_R12M', '할부건수_무이자_6M_R12M', '할부건수_무이자_12M_R12M'

In [106]:
del num_cols[0]

In [108]:
print(num_cols)

['이용가맹점수', '이용금액_해외', '쇼핑_도소매_이용금액', '쇼핑_백화점_이용금액', '쇼핑_마트_이용금액', '쇼핑_슈퍼마켓_이용금액', '쇼핑_편의점_이용금액', '쇼핑_아울렛_이용금액', '쇼핑_온라인_이용금액', '쇼핑_기타_이용금액', '교통_주유이용금액', '교통_정비이용금액', '교통_통행료이용금액', '교통_버스지하철이용금액', '교통_택시이용금액', '교통_철도버스이용금액', '여유_운동이용금액', '여유_Pet이용금액', '여유_공연이용금액', '여유_공원이용금액', '여유_숙박이용금액', '여유_항공이용금액', '여유_기타이용금액', '납부_통신비이용금액', '납부_관리비이용금액', '납부_가스전기료이용금액', '납부_보험료이용금액', '납부_기타이용금액', '_1순위업종_이용금액', '_2순위업종_이용금액', '_3순위업종_이용금액', '_1순위쇼핑업종_이용금액', '_2순위쇼핑업종_이용금액', '_3순위쇼핑업종_이용금액', '_1순위교통업종_이용금액', '_2순위교통업종_이용금액', '_3순위교통업종_이용금액', '_1순위여유업종_이용금액', '_2순위여유업종_이용금액', '_3순위여유업종_이용금액', '_1순위납부업종_이용금액', '_2순위납부업종_이용금액', '_3순위납부업종_이용금액', '할부건수_3M_R12M', '할부건수_6M_R12M', '할부건수_12M_R12M', '할부건수_14M_R12M', '할부금액_3M_R12M', '할부금액_6M_R12M', '할부금액_12M_R12M', '할부금액_14M_R12M', '할부건수_유이자_3M_R12M', '할부건수_유이자_6M_R12M', '할부건수_유이자_12M_R12M', '할부건수_유이자_14M_R12M', '할부금액_유이자_3M_R12M', '할부금액_유이자_6M_R12M', '할부금액_유이자_12M_R12M', '할부금액_유이자_14M_R12M', '할부건수_무이자_3M_R12M', '할부건수_무이자_6M_R12M', '할부건수_무이자_12M_R12M', '할부건수_

In [130]:
# 제외할 컬럼 리스트
remove_cols = ['여유_여행이용금액', '납부_렌탈료이용금액', '납부_유선방송이용금액', 
               '납부_건강연금이용금액', '할부건수_부분_3M_R12M', '할부건수_부분_6M_R12M', '할부건수_부분_14M_R12M', '할부금액_부분_3M_R12M']

# 기존 숫자형 컬럼 리스트에서 제거
num_cols = [col for col in num_cols if col not in remove_cols]

In [132]:
print(num_cols)

['이용가맹점수', '이용금액_해외', '쇼핑_도소매_이용금액', '쇼핑_백화점_이용금액', '쇼핑_마트_이용금액', '쇼핑_슈퍼마켓_이용금액', '쇼핑_편의점_이용금액', '쇼핑_아울렛_이용금액', '쇼핑_온라인_이용금액', '쇼핑_기타_이용금액', '교통_주유이용금액', '교통_정비이용금액', '교통_통행료이용금액', '교통_버스지하철이용금액', '교통_택시이용금액', '교통_철도버스이용금액', '여유_운동이용금액', '여유_Pet이용금액', '여유_공연이용금액', '여유_공원이용금액', '여유_숙박이용금액', '여유_항공이용금액', '여유_기타이용금액', '납부_통신비이용금액', '납부_관리비이용금액', '납부_가스전기료이용금액', '납부_보험료이용금액', '납부_기타이용금액', '_1순위업종_이용금액', '_2순위업종_이용금액', '_3순위업종_이용금액', '_1순위쇼핑업종_이용금액', '_2순위쇼핑업종_이용금액', '_3순위쇼핑업종_이용금액', '_1순위교통업종_이용금액', '_2순위교통업종_이용금액', '_3순위교통업종_이용금액', '_1순위여유업종_이용금액', '_2순위여유업종_이용금액', '_3순위여유업종_이용금액', '_1순위납부업종_이용금액', '_2순위납부업종_이용금액', '_3순위납부업종_이용금액', '할부건수_3M_R12M', '할부건수_6M_R12M', '할부건수_12M_R12M', '할부건수_14M_R12M', '할부금액_3M_R12M', '할부금액_6M_R12M', '할부금액_12M_R12M', '할부금액_14M_R12M', '할부건수_유이자_3M_R12M', '할부건수_유이자_6M_R12M', '할부건수_유이자_12M_R12M', '할부건수_유이자_14M_R12M', '할부금액_유이자_3M_R12M', '할부금액_유이자_6M_R12M', '할부금액_유이자_12M_R12M', '할부금액_유이자_14M_R12M', '할부건수_무이자_3M_R12M', '할부건수_무이자_6M_R12M', '할부건수_무이자_12M_R12M', '할부건수_

In [134]:
anova = []
chi = []

for col in all_df.columns:
    if col == 'Segment':
        continue

    if col in num_cols:
        data = all_df[[col, 'Segment']].dropna()
        groups = [data[data['Segment'] == val][col] for val in data['Segment'].unique()]
        try:
            stat = f_oneway(*groups).statistic
            ss_between = sum([(g.mean() - data[col].mean())**2 * len(g) for g in groups])
            ss_total = sum((data[col] - data[col].mean())**2)
            eta2 = eta_squared(ss_between, ss_total)
            anova.append({'변수': col, '유형': '수치형', '계수종류': 'Eta²', '상관계수': eta2})
        except:
            continue

    elif col in cat_cols:
        contingency = pd.crosstab(all_df[col], all_df['Segment'])
        if contingency.shape[0] > 1 and contingency.shape[1] > 1:
            try:
                v = cramers_v(contingency)
                chi.append({'변수': col, '유형': '범주형', '계수종류': "Cramér's V", '상관계수': v})
            except:
                continue

# 결과 정리
result_df1 = pd.DataFrame(anova)
result_df2 = pd.DataFrame(chi)
result_df1 = result_df1.sort_values(by='상관계수', ascending=False).reset_index(drop=True)
result_df2 = result_df2.sort_values(by='상관계수', ascending=False).reset_index(drop=True)

# 결과 출력
display(result_df1)
display(result_df2)

,변수,유형,계수종류,상관계수
0,_3순위업종_이용금액,수치형,Eta²,2.549372e-01
1,_2순위업종_이용금액,수치형,Eta²,2.429217e-01
2,_2순위쇼핑업종_이용금액,수치형,Eta²,2.387084e-01
3,_1순위업종_이용금액,수치형,Eta²,2.198912e-01
4,_3순위쇼핑업종_이용금액,수치형,Eta²,2.172726e-01
5,이용가맹점수,수치형,Eta²,2.157008e-01
6,쇼핑_도소매_이용금액,수치형,Eta²,2.048195e-01
7,_1순위교통업종_이용금액,수치형,Eta²,1.715343e-01
8,쇼핑_마트_이용금액,수치형,Eta²,1.575289e-01
9,쇼핑_슈퍼마켓_이용금액,수치형,Eta²,1.560192e-01


,변수,유형,계수종류,상관계수
0,_2순위여유업종,범주형,Cramér's V,0.131715
1,_1순위업종,범주형,Cramér's V,0.116062
2,_3순위여유업종,범주형,Cramér's V,0.105205
3,_2순위업종,범주형,Cramér's V,0.102366
4,_3순위쇼핑업종,범주형,Cramér's V,0.097754
5,_1순위쇼핑업종,범주형,Cramér's V,0.097296
6,_3순위업종,범주형,Cramér's V,0.091734
7,_1순위여유업종,범주형,Cramér's V,0.085033
8,_1순위교통업종,범주형,Cramér's V,0.084260
9,_2순위쇼핑업종,범주형,Cramér's V,0.080923


In [142]:
pd.set_option('display.max_rows', 100)
display(result_df1)

,변수,유형,계수종류,상관계수
0,_3순위업종_이용금액,수치형,Eta²,2.549372e-01
1,_2순위업종_이용금액,수치형,Eta²,2.429217e-01
2,_2순위쇼핑업종_이용금액,수치형,Eta²,2.387084e-01
3,_1순위업종_이용금액,수치형,Eta²,2.198912e-01
4,_3순위쇼핑업종_이용금액,수치형,Eta²,2.172726e-01
5,이용가맹점수,수치형,Eta²,2.157008e-01
6,쇼핑_도소매_이용금액,수치형,Eta²,2.048195e-01
7,_1순위교통업종_이용금액,수치형,Eta²,1.715343e-01
8,쇼핑_마트_이용금액,수치형,Eta²,1.575289e-01
9,쇼핑_슈퍼마켓_이용금액,수치형,Eta²,1.560192e-01


In [150]:
result_df1['상관계수'] = result_df1['상관계수'].round(4)

In [152]:
display(result_df1)

,변수,유형,계수종류,상관계수
0,_3순위업종_이용금액,수치형,Eta²,0.2549
1,_2순위업종_이용금액,수치형,Eta²,0.2429
2,_2순위쇼핑업종_이용금액,수치형,Eta²,0.2387
3,_1순위업종_이용금액,수치형,Eta²,0.2199
4,_3순위쇼핑업종_이용금액,수치형,Eta²,0.2173
5,이용가맹점수,수치형,Eta²,0.2157
6,쇼핑_도소매_이용금액,수치형,Eta²,0.2048
7,_1순위교통업종_이용금액,수치형,Eta²,0.1715
8,쇼핑_마트_이용금액,수치형,Eta²,0.1575
9,쇼핑_슈퍼마켓_이용금액,수치형,Eta²,0.1560


### corr 상관계수

In [112]:
# all_df에서 기준년월, ID 컬럼은 제거한다. 
all_df = all_df.drop(['기준년월','ID'], axis=1)

In [113]:
cat_cols

['Segment',
 '_1순위업종',
 '_2순위업종',
 '_3순위업종',
 '_1순위쇼핑업종',
 '_2순위쇼핑업종',
 '_3순위쇼핑업종',
 '_1순위교통업종',
 '_2순위교통업종',
 '_3순위교통업종',
 '_1순위여유업종',
 '_2순위여유업종',
 '_3순위여유업종',
 '_1순위납부업종',
 '_2순위납부업종',
 '_3순위납부업종']

In [135]:
# A가 가장 높은 등급일 것이라 추측하고 세그먼트만 LabelEncoder가 아닌 map으로 진행

segment_order = {'E': 0, 'D': 1, 'C': 2, 'B': 3, 'A': 4}
all_df['Segment_e'] = all_df['Segment'].map(segment_order)

In [ ]:
# 인코딩 진행
# 인코딩할 컬럼만 선택
encoder = LabelEncoder()
all_df[''] = encoder.fit_transform(all_df[''])
all_df[''] = encoder.fit_transform(all_df[''])
all_df[''] = encoder.fit_transform(all_df[''])
all_df[''] = encoder.fit_transform(all_df[''])
all_df[''] = encoder.fit_transform(all_df[''])

In [136]:
print(all_df['Segment_e'].unique())

[1 0 2 4 3]


In [137]:
# 비교할 종속변수
target = 'Segment_e'
# 결과를 담을 리스트 
corr_result = []
# 반복
for col in num_cols:
    if col in all_df.columns:
        corr = all_df[col].corr(all_df[target])
        corr_result.append({'변수': col, '상관계수': corr})

# 데이터프레임으로 정리한다 
result_df3 = pd.DataFrame(corr_result).sort_values(by='상관계수', key=abs, ascending=False).reset_index(drop=True)

display(result_df3)

,변수,상관계수
0,_3순위업종_이용금액,0.504268
1,_2순위업종_이용금액,0.492209
2,_2순위쇼핑업종_이용금액,0.487364
3,_1순위업종_이용금액,0.468099
4,_3순위쇼핑업종_이용금액,0.464139
5,이용가맹점수,0.456553
6,쇼핑_도소매_이용금액,0.452097
7,_1순위교통업종_이용금액,0.408528
8,쇼핑_마트_이용금액,0.395007
9,쇼핑_슈퍼마켓_이용금액,0.390990


In [154]:
result_AK = pd.concat(
    [result_df1, result_df2],
    axis=0,
    ignore_index=True
)
display(result_AK)

,변수,유형,계수종류,상관계수
0,_3순위업종_이용금액,수치형,Eta²,0.254900
1,_2순위업종_이용금액,수치형,Eta²,0.242900
2,_2순위쇼핑업종_이용금액,수치형,Eta²,0.238700
3,_1순위업종_이용금액,수치형,Eta²,0.219900
4,_3순위쇼핑업종_이용금액,수치형,Eta²,0.217300
5,이용가맹점수,수치형,Eta²,0.215700
6,쇼핑_도소매_이용금액,수치형,Eta²,0.204800
7,_1순위교통업종_이용금액,수치형,Eta²,0.171500
8,쇼핑_마트_이용금액,수치형,Eta²,0.157500
9,쇼핑_슈퍼마켓_이용금액,수치형,Eta²,0.156000


In [156]:
### 내보낸다. 
result_AK.to_excel('승인매출정보3-2_상관계수.xlsx', index=False)
result_df3.to_excel('승인매출정보3-2_검사.xlsx', index=False)


### VIF

In [23]:
# 사용할 수치형 변수 리스트
features = ['최초한도금액', '카드이용한도금액',
       'CA한도금액', 'CA이자율_할인전', '카드이용한도금액_B1M',
       '카드이용한도금액_B2M']
features1 = ['최초한도금액',
       'CA이자율_할인전',
       'CA한도금액']
features2 = ['최초한도금액', '카드이용한도금액',
       'CA한도금액', 'CA이자율_할인전', '카드이용한도금액_B2M']
# 독립변수 데이터
X = all_df[features2]
X = add_constant(X)  # 상수항 추가

# VIF 계산
vif_df = pd.DataFrame()
vif_df['Feature'] = X.columns
vif_df['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_df)

        Feature        VIF
0         const  54.048339
1        최초한도금액   1.019199
2      카드이용한도금액  19.091935
3        CA한도금액   7.152437
4     CA이자율_할인전   1.064573
5  카드이용한도금액_B2M  16.313782


In [24]:
all_df[column5].corr()

,한도증액횟수_R12M,한도증액금액_R12M,한도증액후경과월,상향가능한도금액,상향가능CA한도금액,Segment_e
한도증액횟수_R12M,1.000000,0.941650,-0.819507,0.032296,0.034784,0.060724
한도증액금액_R12M,0.941650,1.000000,-0.763369,0.036540,0.040480,0.084180
한도증액후경과월,-0.819507,-0.763369,1.000000,0.003790,0.001530,-0.039390
상향가능한도금액,0.032296,0.036540,0.003790,1.000000,0.963278,0.050764
상향가능CA한도금액,0.034784,0.040480,0.001530,0.963278,1.000000,0.064575
Segment_e,0.060724,0.084180,-0.039390,0.050764,0.064575,1.000000


### CramersV

In [26]:
# 범주형 vs 범주형: Cramér's V
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    if confusion_matrix.shape[0] < 2 or confusion_matrix.shape[1] < 2:
        return np.nan  # 최소 2x2 이상 필요
    try:
        chi2 = chi2_contingency(confusion_matrix)[0]
        n = confusion_matrix.sum().sum()
        phi2 = chi2 / n
        r, k = confusion_matrix.shape
        return np.sqrt(phi2 / min(k - 1, r - 1))
    except:
        return np.nan


In [27]:
combined_categorical_cols = [
    '자발한도감액횟수_R12M',
    '자발한도감액금액_R12M',
    '자발한도감액후경과월',
    '강제한도감액횟수_R12M',
    '강제한도감액금액_R12M',
    '강제한도감액후경과월',
    '카드론동의여부',
    '월상환론상향가능한도금액',
    'RV전환가능여부',
    '일시불ONLY전환가능여부',
    '특별한도보유여부_R3M',
    '연체감액여부_R3M',
    '한도증액횟수_R12M',
    '한도증액금액_R12M',
    '한도증액후경과월',
    '상향가능한도금액',
    '상향가능CA한도금액',
    '한도심사요청건수',
    '한도요청거절건수',
    '한도심사요청후경과월',
    '한도심사거절후경과월',
    '시장단기연체여부_R6M',
    '시장단기연체여부_R3M',
    '시장연체상환여부_R6M',
    'rv최초시작후경과일'
]


numerical_cols = ['최초한도금액',
 '카드이용한도금액',
 'CA한도금액',
 '일시상환론한도금액',
 '월상환론한도금액',
 'CA이자율_할인전',
 'CL이자율_할인전','RV일시불이자율_할인전',
 'RV현금서비스이자율_할인전',
 'RV약정청구율',
 'RV최소결제비율',
 '카드이용한도금액_B1M',
 '카드이용한도금액_B2M']

In [28]:
# 변수 정의
segment_col = 'Segment'
categorical_cols = combined_categorical_cols  # Segment_e 제외된 범주형 컬럼 리스트

results = []

for col in categorical_cols:
    if col == segment_col:
        continue

    coeff = cramers_v(all_df[segment_col], all_df[col])
    coef_type = "Cramér's V"
    var_type = "범주형"

    results.append({
        '변수': col,
        '유형': var_type,
        '계수종류': coef_type,
        '상관계수': round(coeff, 6) if not pd.isna(coeff) else np.nan
    })

correlation_df = pd.DataFrame(results).sort_values(by='상관계수', ascending=False)
display(correlation_df)



,변수,유형,계수종류,상관계수
15,상향가능한도금액,범주형,Cramér's V,0.141612
16,상향가능CA한도금액,범주형,Cramér's V,0.136274
13,한도증액금액_R12M,범주형,Cramér's V,0.078676
24,rv최초시작후경과일,범주형,Cramér's V,0.070303
12,한도증액횟수_R12M,범주형,Cramér's V,0.070096
8,RV전환가능여부,범주형,Cramér's V,0.044128
6,카드론동의여부,범주형,Cramér's V,0.038021
5,강제한도감액후경과월,범주형,Cramér's V,0.029208
4,강제한도감액금액_R12M,범주형,Cramér's V,0.028309
14,한도증액후경과월,범주형,Cramér's V,0.027772


모든 조합이 True라면 Segment마다 평균이 모두 달라 매우 유의미한 변수